In [ ]:
import ROOT
import SndlhcGeo
import shipunit as u

In [ ]:
geo = SndlhcGeo.GeoInterface("/eos/experiment/sndlhc/convertedData/physics/2024/run_245/geofile_sndlhc_TI18_V5_2024.root")

The detector *objects* are accessible through the ROOT's ListOfGlobals.

In [ ]:
lsOfGlobals = ROOT.gROOT.GetListOfGlobals()
lsOfGlobals.FindObject('Scifi')
lsOfGlobals.FindObject('MuFilter')

SND detectors can be *read* as python dictionaries thanks to the SndlhcGeo module.

In [ ]:
scifi = geo.snd_geo.Scifi
mufi= geo.snd_geo.MuFilter

In [ ]:
print("Size of a Veto3 bar {}, {}, {}".format(mufi.Veto3BarX/u.cm, mufi.Veto3BarY/u.cm, mufi.Veto3BarZ/u.cm))

Why is the Y coordinate the largest 46.0 cm?
Which is the coordinate system, where dimensions are defined? 
The survey or the physics one?
Physics coordinate system is used to store the detector attributes - sizes etc.
Survey CS is used when measuring the detector positions, then the transformation from survey to the physics CS is applied in the sw!
And the user doesn't have to make this transformation by hand.

Inspect the contents of the detector objects, see what keys there exist.
Keys featuring 4 digits( the run numbers) denote the spatial and time alignment constants!

In [ ]:
scifi.keys()

In [ ]:
scifi.Xpos0, scifi.Ypos0, scifi.Zpos0# survey CS

In [ ]:
mufi.keys() # items in the MuFilter detector object

In [ ]:
mufi.Muon1Dx, mufi.Muon1Dy, mufi.Muon1Dz # US1

In [ ]:
mufi.Muon9Dx, mufi.Muon9Dy, mufi.Muon9Dz #DS4

Lets now see how to access the detector elements through the measurements  - hits or clusters

In [ ]:
f = ROOT.TFile("/eos/experiment/sndlhc/convertedData/physics/2024/run_245/run_009280/sndsw_raw-0010.root")
tree = f.rawConv
# provide the event header to the detector object so to read the aligned geometry!
tree.GetEvent(0)
scifi_det=geo.modules['Scifi']
mufi_det=geo.modules['MuFilter']
scifi_det.InitEvent(tree.EventHeader)
mufi_det.InitEvent(tree.EventHeader)
# beware it is not scifi.InitEvent(tree.EventHeader)
# as scifi is just a dictionary!

A,B=ROOT.TVector3(),ROOT.TVector3()

for i, event in enumerate(tree):
    if i >10: continue
    for aHit in event.Digi_ScifiHits:
        scifi_det.GetSiPMPosition(aHit.GetDetectorID(),A,B) # A is left(top) side and the B right/bottom one
        #GetSiPMPosition returns coordinates in the physics CS
        #A.Print()
    for aHit in event.Digi_MuFilterHits:
        mufi_det.GetPosition(aHit.GetDetectorID(),A,B) # A is left(top) side and the B right/bottom one
        A.Print()


Add a task to make Scifi clusters and inspect their positions
The task is actually the simple tracking (SndlhcTracking module)
We need the FairRun

In [ ]:
mufi_det.GetPosition(30060, A,B)# bottom bar, left
A.Print()
B.Print()

mufi_det.GetPosition(30119, A,B)# top bar, right
A.Print()
B.Print()

In [ ]:
import SndlhcTracking
trackTask = SndlhcTracking.Tracking()

In [ ]:
run = ROOT.FairRunAna()
# Input/output manager
ioman = ROOT.FairRootManager.Instance()
source = ROOT.FairFileSource(f)
ioman.SetSource(source)
#run.SetSource(source)
outFile = ROOT.TMemFile('dummy','CREATE')
sink = ROOT.FairRootFileSink(outFile)
ioman.SetSink(sink)
#run.SetSink(sink)
# Add all task you'd like to run
run.AddTask(trackTask)
# Initialize the task collection(Fair run)
run.Init()

#avoiding some error messages
xrdb = ROOT.FairRuntimeDb.instance()
xrdb.getContainer("FairBaseParSet").setStatic()
xrdb.getContainer("FairGeoParSet").setStatic()

In [ ]:
for i, event in enumerate(ioman.GetInTree()):
    if i>10: break
    trackTask.clusScifi.Clear()
    trackTask.scifiCluster()# call the clusterization, clusters will be saved in trackTask.clusScifi
    print("Event number", i)
    for n, aCluster in enumerate(trackTask.clusScifi):
        print("cluster", n, aCluster.GetN())
        # Get position of the cluster.
        # One uses aCluster.GetPosition that returns average XYZ coordinates over hits used to make this cluster
        aCluster.GetPosition(A, B)# no detID needed here!
        print("cluster {}, {}, {}".format(A.x(),A.y(),A.z()))
        # lets see the position of the hit having the fastest signal in this cluster of hits
        scifi_det.GetSiPMPosition(aCluster.GetFirst(),A,B) # 
        print(aCluster.GetFirst())
        print("fast hit A(left/top) {}, {}, {}".format(A.x(),A.y(),A.z()))
        print("fast hit B(right/bottom) {}, {}, {}".format(B.x(),B.y(),B.z()))